Pau Ruiz 217962
<br> Paula Sarrà 216886
<br> Jordi Valsells 218862

# Project IR-WA

#### Load Python packages
Let's first import all the packages that you will need during this assignment.

In [1]:
# if you do not have 'nltk', the following command should work "python -m pip install nltk"
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pau\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import time
import re
import json
import random
import pandas as pd
import collections
import matplotlib.pyplot as plt
from collections import Counter

## PART 1: Text Processing

#### Load data into memory
The dataset is stored in the TSV file, and it contains 500 Wikipedia articles (one article per line). For each article we have the document id, document title and document body separated by "|" character.

In [3]:
#Path of the document
docs_path = 'inputs/dataset_tweets_WHO.txt'

# String of path to file: tweets_data_path
tweets_data_path = 'inputs/dataset_tweets_WHO.txt'

# Open connection to file
tweets_file = open(tweets_data_path, "r")

# Read in tweets and store in list: tweets_data
for line in tweets_file:
    tweets_data_raw = json.loads(line)

# Close connection to file
tweets_file.close()

#Change str keys to int
tweets_data = {int(key):value for key, value in tweets_data_raw.items()}

# Print first tweet of the dict
#print(tweets_data[0])

#Total number of tweets
print('\nTotal number of tweets: {}'.format(len(tweets_data)))


Total number of tweets: 2399


In [4]:
#print a tweet
tweets_data[0]

{'created_at': 'Wed Oct 13 09:15:58 +0000 2021',
 'id': 1448215930178310144,
 'id_str': '1448215930178310144',
 'full_text': "It's International Day for Disaster Risk Reduction\n\n#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.\n\nStart learning today &amp; be #Ready4Response:\n👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS",
 'truncated': False,
 'display_text_range': [0, 274],
 'entities': {'hashtags': [{'text': 'OpenWHO', 'indices': [52, 60]},
   {'text': 'Ready4Response', 'indices': [232, 247]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/hBFFOF0xKL',
    'expanded_url': 'https://bit.ly/3wCa0Dr',
    'display_url': 'bit.ly/3wCa0Dr',
    'indices': [251, 274]}],
  'media': [{'id': 1448215398814560259,
    'id_str': '1448215398814560259',
    'indices': [275, 298],
    'media_url': 'http://pbs.twimg.com/ext_tw_video_thumb/1448215398814560259/pu/img/0CO

In [5]:
def build_terms(line):
    """
    Preprocess the tweet (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    hashtags - a list of hashtags that the text contained
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    
    line = line.lower() ## Transform in lowercase
    line = re.sub(r'http\S+', '', line) ## remove links
    line = re.sub(r'[^A-Za-z0-9#\n ]+', '', line) ## remove special characters (not # yet)
    line = line.split() ## Tokenize the text to get a list of terms
    
    hashtags=[]
    
    for word in range(len(line)):
        if line[word][0]=='#':
            line[word]=line[word].replace('#', '') ## now remove # too
            hashtags.append(line[word]) ## and add to the list of hashtags
            
    line = [word for word in line if word not in stop_words]  ##eliminate the stopwords
    line = [stemmer.stem(word) for word in line] ## perform stemming
    
    return line, hashtags

In [6]:
length=len(tweets_data)
hashtags={}

#we apply our function build terms to the tweets dataset and also save the hashtags in a list
for key in range(length):
    tweets_data[key]['full_text'], hashtags[tweets_data[key]['id']] = build_terms(tweets_data[key]['full_text'])

## PART 2: Indexing and Evaluation

## Indexing

### Inverted index

In [7]:
def create_index(tweets_data):
    """
    Implement the inverted index
    
    Argument:
    lines -- collection of tweets 
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of tweets where these keys appears in (and the positions) as values.
    """
    index = defaultdict(list)
    title_index = {}
    for tweet in tweets_data:  # Remember, lines contain all documents
        tweet_id = int(tweets_data[tweet]['id'])
        terms =  tweets_data[tweet]['full_text']#page_title + page_text
        title = " ".join(tweets_data[tweet]['full_text'])
        title_index[tweet_id]=title
        
        ## ===============================================================        
        ## create the index for the current page and store it in current_page_index (current_page_index)
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
                
        ## START CODE
                current_page_index[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[tweet_id, array('I',[position])] #'I' indicates unsigned int (int in Python)
            
        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
        
        ## END CODE                    
                    
    return index, title_index

In [8]:
start_time = time.time()
index, title_index = create_index(tweets_data)
print("Total time to create the index: {} seconds".format(np.round(time.time() - start_time, 2)))

Total time to create the index: 0.09 seconds


In [9]:
def search(query, index):
    """
    The output is the list of tweets that contain any of the query terms. 
    So, we will get the list of tweets for each query term, and take the union of them.
    """
    query, hashtags = build_terms(query)
    docs = set()
    for term in query:
    ## START DODE
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=[posting[0] for posting in index[term]]            
            # docs = docs Union term_docs
            docs |= set(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    return docs

In [10]:
print("Insert your query:\n")

query = input()
docs = search(query, index)
top = 10

print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(docs)))
for d_id in docs[:top]:
    print("\n TWEET= {} - tweet_id: {}".format(title_index[d_id],d_id))    

Insert your query:

covid vaccine

Sample of 10 results out of 436 for the searched query:


 TWEET= rt whoafro congratul algeria algeria 16th countri africa reach mileston fulli vaccin 10 pop - tweet_id: 1448163447678676992

 TWEET= clinic case definit post covid19 condit also call long covid - tweet_id: 1446089970758860801

 TWEET= rt whophilippin vaccin cant stop covid19 alon help protect love one covid - tweet_id: 1447421491428143106

 TWEET= learn first malaria vaccin receiv vaccin challeng roll malaria vaccin midst pandem dr palonsomalaria explain sciencein5 week - tweet_id: 1446601142373425153

 TWEET= rt drtedro thank much govern peopl thailand solidar continu financi support covid - tweet_id: 1442947527716704260

 TWEET= recov covid19 still experienc certain symptom could post covid19 condit long covid symptom long last treatment option dr diazjv explain sciencein5 - tweet_id: 1422159909957869572

 TWEET= rt drtedro must appreci role privat sector play covid19 respons develop 

### 5 Queries

In [11]:
def find_keywords():
    """
    Find the most repeated words in the dataset.
    """
    wordcount = {}
    for key in range(length):
        words = tweets_data[key]['full_text']
        for word in words:
            if word not in wordcount:
                wordcount[word] = 1
            else:
                wordcount[word] += 1
    n_print = int(input("How many most common words to print: "))
    print("\nOK. The {} most common words are as follows\n".format(n_print))
    word_counter = collections.Counter(wordcount)
    for word, count in word_counter.most_common(n_print):
        print(word, ": ", count)
find_keywords()

How many most common words to print: 10

OK. The 10 most common words are as follows

amp :  1030
drtedro :  979
health :  774
covid19 :  759
rt :  549
vaccin :  545
countri :  395
peopl :  353
support :  271
global :  254


### TF-IDF

In [12]:
def create_index_tfidf(tweets_data, num_tweets):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of tweets
    num_documents -- total number of tweets
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """

    index = defaultdict(list)
    tf = defaultdict(list)  #term frequencies of terms in tweets (tweets in the same order as in the main index)
    df = defaultdict(int)  #tweet frequencies of terms in the corpus
    title_index = defaultdict(str)
    idf = defaultdict(float)

    for tweet in tweets_data:
        
        tweet_id = int(tweets_data[tweet]['id'])
        terms =  tweets_data[tweet]['full_text']#page_title + page_text
        title = " ".join(tweets_data[tweet]['full_text'])
        title_index[tweet_id]=title
        ## ===============================================================        
        ## create the index for the **current page** and store it in current_page_index
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms):  ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corresponding list
                current_page_index[term][1].append(position)
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[tweet_id, array('I',[position])] #'I' indicates unsigned int (int in Python)

        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm = 0
        for term, posting in current_page_index.items():
            # posting will contain the list of positions for current term in current document. 
            # posting ==> [current_doc, [list of positions]] 
            # you can use it to infer the frequency of current term.
            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)

        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4)) ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term] +=1 # increment DF for current term

        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)

        # Compute IDF following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round(np.log(float(num_tweets/df[term])), 4)

    return index, tf, df, idf, title_index

In [13]:
start_time = time.time()
num_tweets = len(tweets_data)
index, tf, df, idf, title_index = create_index_tfidf(tweets_data, num_tweets)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to create the index: 67.39 seconds


In [14]:
def rank_documents(terms, docs, index, idf, tf, title_index):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    title_index -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """

    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    #HINT: use when computing tf for query_vector

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex]=query_terms_count[term]/query_norm * idf[term] 

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):
            # Example of [doc_index, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....

            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  # TODO: check if multiply for idf

    # Calculate the score of each tweet 
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine similarity
    # see np.dot
    
    doc_scores=[[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores_df = [x[0] for x in doc_scores]
    docs_id_df = [x[1] for x in doc_scores]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    result_doc_scores = [x[0] for x in doc_scores]
    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]
    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)
    #print ('\n'.join(result_docs), '\n')
    
    return result_docs, doc_scores_df, docs_id_df

In [15]:
def search_tf_idf(query, index):
    """
    output is the list of tweets that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query, hashtags = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=[posting[0] for posting in index[term]]
            
            # docs = docs Union term_docs
            docs |= set(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    ranked_docs, doc_scores_df, docs_id_df = rank_documents(query, docs, index, idf, tf, title_index)
    
    return ranked_docs, doc_scores_df, docs_id_df

## Returning ranking with TF-IDF + cosine similarity:

In [16]:
our_queries=['covid19 vaccine', 'pandemic', 'health access', 'countries support', 'people care']

queries_df = pd.DataFrame()
for i in range(5):
    query = our_queries[i]
    ranked_docs, doc_scores_df, docs_id_df = search_tf_idf(query, index)
    
    hashtag_found = [0]*len(docs_id_df)
    
    for word in query.split():
        for doc in range(len(docs_id_df)):
            for h in hashtags[docs_id_df[doc]]:
                #print(h)
                if h == word:
                    hashtag_found[doc] = 1 #as it is binary
    if i == 0:
        queries_df = pd.DataFrame()
        queries_df['doc_id'] = docs_id_df
        queries_df['q_id'] = i
        queries_df['predicted_relevance'] = doc_scores_df
        queries_df['y_true'] = hashtag_found
    else:
        #print("here")
        query_df = pd.DataFrame()
        query_df['doc_id'] = docs_id_df
        query_df['q_id'] = i
        query_df['predicted_relevance'] = doc_scores_df
        query_df['y_true'] = hashtag_found
        queries_df = pd.concat([queries_df, query_df], ignore_index=True)
    
    top = 20
    print("\n", query)
    print("======================\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
    for d_id in ranked_docs[:top]:
        print("tweet_id= {} - tweet: {}".format(d_id, title_index[d_id]))


 covid19 vaccine
Top 20 results out of 887 for the searched query:

tweet_id= 1418489081026154497 - tweet: mrna covid19 vaccin safe vaccin read
tweet_id= 1432284372690866186 - tweet: covid19 vaccin could save life get vaccin soon turn
tweet_id= 1426206173225787392 - tweet: rt whowpro encourag love one get vaccin covid19 help regist get vaccin site vaccin
tweet_id= 1418545630494924805 - tweet: covid19 vaccin halal read
tweet_id= 1416433609091653633 - tweet: covid19 vaccin covid19 vaccin 10 countri rest vaccinequ end pandem togeth worldemojiday
tweet_id= 1408416636084707335 - tweet: get vaccin even covid19
tweet_id= 1430183353638998018 - tweet: facilit effect vaccin deliveri person disabl health provid provid access covid19 vaccin info amp vaccin process ensur access vaccin site vaccinequ
tweet_id= 1439194751538974723 - tweet: rt drtedro glad thank covid19 vaccin walk vaccin process omanconvent storag
tweet_id= 1437865493118005248 - tweet: who global covid19 vaccin target 70 popul count

In [17]:
queries_df

,doc_id,q_id,predicted_relevance,y_true
0,1448208458604584960,0,0.201941,1
1,1448163383493136385,0,0.177700,0
2,1448049654923468803,0,0.849441,1
3,1448031156348362754,0,0.285646,1
4,1448031115428696064,0,0.264472,1
...,...,...,...,...
2766,1409489891797446656,4,0.834446,0
2767,1408450795570335747,4,1.152974,0
2768,1408450214139240452,4,1.274653,0
2769,1408449858365710339,4,0.956125,0


## Our score

In [18]:
#The favorite_count provides the number of times the tweet has been favorited.
#The retweet_count is the count of the retweets of the source tweet

In [19]:
tweet_id = int(tweets_data[0]['id'])
terms =  tweets_data[0]['full_text'] #page_title + page_text
title = " ".join(tweets_data[0]['full_text'])
title_index[0]=title
num_retweets = tweets_data[0]['retweet_count'] #num retweets
favourite_count = tweets_data[0]['favorite_count'] #num of favourite

print('Tweet id:', tweet_id)
print('Terms:', terms)
print('Title:', title)
print('Number of retweets:', num_retweets)
print('Number of favorites:', favourite_count)

Tweet id: 1448215930178310144
Terms: ['intern', 'day', 'disast', 'risk', 'reduct', 'openwho', 'launch', 'multiti', 'core', 'curriculum', 'help', 'equip', 'compet', 'need', 'work', 'within', 'public', 'health', 'emerg', 'respons', 'start', 'learn', 'today', 'amp', 'ready4respons']
Title: intern day disast risk reduct openwho launch multiti core curriculum help equip compet need work within public health emerg respons start learn today amp ready4respons
Number of retweets: 16
Number of favorites: 52


In [20]:
def create_index_our_score(tweets_data, num_tweets):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of tweets
    num_documents -- total number of tweets
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    popularity - parameter of popularity depending on number of retweets + favourites
    """

    index = defaultdict(list)
    tf = defaultdict(list)  #term frequencies of terms in tweets (tweets in the same order as in the main index)
    df = defaultdict(int)  #tweet frequencies of terms in the corpus
    title_index = defaultdict(str)
    idf = defaultdict(float)
    popularity = []

    for tweet in tweets_data:
        
        tweet_id = int(tweets_data[tweet]['id'])
        terms =  tweets_data[tweet]['full_text']#page_title + page_text
        title = " ".join(tweets_data[tweet]['full_text'])
        title_index[tweet_id]=title
        ## ===============================================================        
        ## create the index for the **current page** and store it in current_page_index
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms):  ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corresponding list
                current_page_index[term][1].append(position)
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[tweet_id, array('I',[position])] #'I' indicates unsigned int (int in Python)

        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm = 0
        for term, posting in current_page_index.items():
            # posting will contain the list of positions for current term in current document. 
            # posting ==> [current_doc, [list of positions]] 
            # you can use it to infer the frequency of current term.
            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)

        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4)) ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term] +=1 # increment DF for current term

        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)

        # Compute IDF following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round(np.log(float(num_tweets/df[term])), 4)
            
        num_retweets = tweets_data[tweet]['retweet_count'] #num retweets
        favourite_count = tweets_data[tweet]['favorite_count'] #num of favourite
        
        popularity.append(num_retweets + favourite_count)
        
        
    popularity = (popularity)/np.linalg.norm(popularity) * 100

    
    return index, tf, df, idf, title_index, popularity

In [21]:
start_time = time.time()
num_tweets = len(tweets_data)
index, tf, df, idf, title_index, popularity = create_index_our_score(tweets_data, num_tweets)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to create the index: 55.79 seconds


In [22]:
def rank_documents_our_score(terms, docs, index, idf, tf, title_index, popularity):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    title_index -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """

    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    #HINT: use when computing tf for query_vector

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex]=query_terms_count[term]/query_norm * idf[term] 

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):
            # Example of [doc_index, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....

            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term] * popularity[doc_index] # TODO: check if multiply for idf

    # Calculate the score of each tweet 
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine similarity
    # see np.dot
    
    doc_scores=[[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores_df = [x[0] for x in doc_scores]
    docs_id_df = [x[1] for x in doc_scores]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    result_doc_scores = [x[0] for x in doc_scores]
    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]
    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)
    #print ('\n'.join(result_docs), '\n')
    
    return result_docs, doc_scores_df, docs_id_df

In [23]:
def search_our_score(query, index):
    """
    output is the list of tweets that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query, hashtags = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=[posting[0] for posting in index[term]]
            
            # docs = docs Union term_docs
            docs |= set(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    ranked_docs, doc_scores_df, docs_id_df = rank_documents_our_score(query, docs, index, idf, tf, title_index, popularity)
    
    return ranked_docs, doc_scores_df, docs_id_df

In [24]:
queries_df = pd.DataFrame()
for i in range(5):
    query = our_queries[i]
    ranked_docs, doc_scores_df, docs_id_df = search_our_score(query, index)
    
    hashtag_found = [0]*len(docs_id_df)
    
    for word in query.split():
        for doc in range(len(docs_id_df)):
            for h in hashtags[docs_id_df[doc]]:
                #print(h)
                if h == word:
                    hashtag_found[doc] = 1 #as it is binary
    if i == 0:
        queries_df = pd.DataFrame()
        queries_df['doc_id'] = docs_id_df
        queries_df['q_id'] = i
        queries_df['predicted_relevance'] = doc_scores_df
        queries_df['y_true'] = hashtag_found
    else:
        #print("here")
        query_df = pd.DataFrame()
        query_df['doc_id'] = docs_id_df
        query_df['q_id'] = i
        query_df['predicted_relevance'] = doc_scores_df
        query_df['y_true'] = hashtag_found
        queries_df = pd.concat([queries_df, query_df], ignore_index=True)
    
    top = 20
    print("\n", query)
    print("======================\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
    for d_id in ranked_docs[:top]:
        print("tweet_id= {} - tweet: {}".format(d_id, title_index[d_id]))


 covid19 vaccine
Top 20 results out of 887 for the searched query:

tweet_id= 1425405666580172803 - tweet: rt live qampa vaccinequ dr mariangelasimao amp dr bruce aylward askwho question covid19 vaccin equiti
tweet_id= 1427991963932975108 - tweet: drtedro actacceler urg jampj urgent priorit distribut vaccin africa consid suppli rich countri alreadi suffici accessdrtedro vaccinequ
tweet_id= 1433070724781154310 - tweet: drtedro covid19 pandem defin crisi time taught world mani pain lessonsdrtedro
tweet_id= 1435598519595843585 - tweet: rt whoafro join tomorrow whoafro live press brief covid19 pandem genom sequenc covid19 variant africa dr
tweet_id= 1438528587783495686 - tweet: rt whoafro whoafro covid19 press brief pandem amp impact covax vaccin suppli challeng speaker includ dr moeti
tweet_id= 1418205159188955137 - tweet: covid19 vaccin inequ undermin econom recoveri undp blavatnikschool dashboard vaccinequ find lowincom countri would add 38b gdp forecast vaccin rate highincom countri
t

## Word2Vec

In [25]:
#Creem Word2Vec Model
import pandas as pd
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string

In [26]:
all_words=[]
for key in range(length):
    all_words+=tweets_data[key]["full_text"]
all_words_unique = np.unique(all_words)
all_words = [all_words]

In [27]:
all_tweets=[]
for key in range(length):
    all_tweets.append(tweets_data[key]["full_text"])

In [28]:
model = Word2Vec(sentences = all_words, min_count=1, workers=4)

C:\Users\Pau\anaconda3\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


In [29]:
queries=[]
for query in our_queries:
    query,_ = build_terms(query)
    queries.append(query)

In [30]:
def tweet_query(tweet, query):
    word_query = []
    for w in all_words_unique:
        a = []
        for q in query:
            a.append(np.dot(model[q], model[w]))
        word_query.append(np.mean(a))

    word_tweet = []
    for w in all_words_unique:
        b = []
        for t in tweet:
            b.append(np.dot(model[t], model[w]))
        word_tweet.append(np.mean(b))
    return np.dot(word_query,word_tweet)

In [31]:
my_tweet=all_tweets[0]
my_query = queries[0]
tweet_query(my_tweet, my_query)

C:\Users\Pau\AppData\Local\Temp/ipykernel_9576/2578313831.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  a.append(np.dot(model[q], model[w]))
C:\Users\Pau\AppData\Local\Temp/ipykernel_9576/2578313831.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  b.append(np.dot(model[t], model[w]))


101168.8

In [32]:
def find_scores(tweets_data,all_tweets, query, k_top):
    tweet_dict={}
    for i in range(len(tweets_data))[0:200]:
        tweet=all_tweets[i]
        tweet_id=tweets_data[i]["id"]
        tweet_dict[tweet_id]=tweet_query(tweet, query)
        
    k = Counter(tweet_dict)
    high = k.most_common(k_top)
    return high

In [33]:
for query in queries:
    print("\nQUERY: ", " ".join(query))
    high = find_scores(tweets_data,all_tweets, query, 10)
    print(high)


QUERY:  covid19 vaccin


C:\Users\Pau\AppData\Local\Temp/ipykernel_9576/2578313831.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  a.append(np.dot(model[q], model[w]))
C:\Users\Pau\AppData\Local\Temp/ipykernel_9576/2578313831.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  b.append(np.dot(model[t], model[w]))


[(1447226874262872067, 149004.4), (1446111322739863577, 148547.84), (1446113678776889344, 140102.89), (1447226893783248898, 138054.45), (1447624956615987206, 136688.84), (1446582377711554569, 133625.47), (1447972516161490944, 133145.19), (1446481486346170377, 131655.25), (1447421375321358341, 129712.27), (1446424624141684738, 128736.46)]

QUERY:  pandem
[(1447226874262872067, 135361.7), (1446111322739863577, 134946.94), (1446113678776889344, 127275.2), (1447226893783248898, 125414.31), (1447624956615987206, 124173.734), (1446582377711554569, 121390.84), (1447972516161490944, 120954.54), (1446481486346170377, 119601.016), (1447421375321358341, 117835.94), (1446424624141684738, 116949.47)]

QUERY:  health access
[(1447226874262872067, 133600.44), (1446111322739863577, 133191.06), (1446113678776889344, 125619.15), (1447226893783248898, 123782.48), (1447624956615987206, 122558.04), (1446582377711554569, 119811.36), (1447972516161490944, 119380.734), (1446481486346170377, 118044.81), (14474